In [ ]:
pip install keras


In [100]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [108]:
lines = pd.read_table(r'C:\Users\Hp\Downloads\d.txt', names=['english', 'deu'])
lines = lines[:3000]
lines.sample(5)

,english,deu
799,I hung up.,Ich habe aufgelegt.
1720,You look really gorgeous.,Du siehst wirklich umwerfend aus.
1309,Can I say something?,Darf ich etwas sagen?
1054,Your breath stinks.,Du hast Mundgeruch.
2004,"Where's Tom? ""Beats me.""",„Wo ist Tom?“ – „Null Ahnung.“


In [109]:
lines.shape

(2931, 2)

In [110]:
lines.english = lines.english.apply(lambda x: x.lower())
lines.deu=lines.deu.apply(lambda x:x.lower())

In [111]:
import re
lines.english = lines.english.apply(lambda x: re.sub("'", '', x)).apply(lambda x: re.sub(",", 'COMMA', x))
lines.deu = lines.deu.apply(lambda x: re.sub("'", '', x)).apply(lambda x: re.sub(",", 'COMMA', x))

In [112]:
import string
exclude = set(string.punctuation)
lines.english = lines.english.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.deu= lines.deu.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [113]:
from string import digits
remove_digits = str.maketrans('', '', digits)
lines.english = lines.english.apply(lambda x: x.translate(remove_digits))
lines.deu = lines.deu.apply(lambda x: x.translate(remove_digits))

In [114]:
lines.sample(10)

,english,deu
0,go,geh
1499,did you like boston,hat ihnen boston gefallen
2511,didnt you see toms face,hast du nicht toms gesicht gesehen
497,im thin,ich bin dünn
182,go away,verkrümele dich
2428,did you change the filter,hast du den filter ausgetauscht
2912,dont tell me tom is dead,tom ist tot sag bloß
985,youre really nice,du bist sehr nett
730,he is old,er ist alt
664,chill out,entspann dich


In [115]:
# applying start and end tokens in korean sentences
lines.deu = lines.deu.apply(lambda x: 'START_' + ' ' + x + ' ' + '_END')
lines.head()

,english,deu
0,go,START_ geh _END
1,hi,START_ hallo _END
2,hi,START_ grüß gott _END
3,run,START_ lauf _END
4,run,START_ lauf _END


In [119]:
# collecting all unique english words to create a vocabulary
all_english_words = set()
for eng in lines.english:
  for word in eng.split():
    if word not in all_english_words:
      all_english_words.add(word)

# collecting all unique korean words to create a vocabulary
all_deu_words = set()
for d in lines.deu:
  for word in d.split():
    if word not in all_deu_words:
      all_deu_words.add(word)

In [120]:
# printing length of words in each language
print('length of english words: ', len(all_english_words))
print('length of deu words: ', len(all_deu_words))

length of english words:  1394
length of deu words:  2126


In [121]:
# getting maximum sentence length of english sentences
length_list = []
for l in lines.english:
  length_list.append(len(l.split(' ')))

max_input_length = np.max(length_list)
print('max_input_length: ', max_input_length)

max_input_length:  8


In [122]:
# getting maximum sentence length of deu sentences
length_list = []
for l in lines.deu:
  length_list.append(len(l.split(' ')))

max_output_length = np.max(length_list)
print('max_output_length: ', max_output_length)

max_output_length:  13


tokenization

In [123]:
# making a list of all input and output words and sorting them out
input_words = sorted(list(all_english_words))
output_words = sorted(list(all_deu_words))
print('all input words: ', input_words)
print('all output words: ', output_words)

#getting total tokens(words) from input and output
num_encoder_tokens = len(all_english_words)
num_decoder_tokens = len(all_deu_words)
print('encoder tokens: ', num_encoder_tokens)
print('decoder tokens: ', num_encoder_tokens)

all input words:  ['a', 'about', 'absolute', 'absorbs', 'accident', 'ache', 'across', 'actress', 'actually', 'actuallyCOMMA', 'add', 'address', 'admitted', 'advance', 'advantage', 'advice', 'afford', 'afraid', 'after', 'again', 'agree', 'ahead', 'aid', 'aim', 'air', 'alcohol', 'aliens', 'alive', 'all', 'allergies', 'allow', 'alls', 'almost', 'alone', 'along', 'already', 'also', 'aluminum', 'always', 'am', 'america', 'an', 'and', 'angry', 'angryCOMMA', 'animal', 'animals', 'annoying', 'another', 'answer', 'any', 'anybody', 'anymore', 'anyone', 'anything', 'anythings', 'anywayCOMMA', 'apartment', 'appearances', 'appeared', 'apples', 'approached', 'approve', 'arabic', 'are', 'arent', 'around', 'arrest', 'arrive', 'arrived', 'art', 'as', 'ask', 'asked', 'asking', 'asleep', 'assistance', 'assistant', 'at', 'ate', 'attack', 'attacked', 'attention', 'attractive', 'australia', 'autograph', 'available', 'avoiding', 'awake', 'away', 'awesome', 'awful', 'b', 'baby', 'back', 'backpack', 'bad', 'ba

In [124]:
# getting index for words as these indexes will behave as words for machine interactions
input_token_index = dict([(word,i) for i,word in enumerate(input_words)])
output_token_index = dict([(word,i) for i,word in enumerate(output_words)])

print('input token index: ', input_token_index)
print('output token index: ', output_token_index)

input token index:  {'a': 0, 'about': 1, 'absolute': 2, 'absorbs': 3, 'accident': 4, 'ache': 5, 'across': 6, 'actress': 7, 'actually': 8, 'actuallyCOMMA': 9, 'add': 10, 'address': 11, 'admitted': 12, 'advance': 13, 'advantage': 14, 'advice': 15, 'afford': 16, 'afraid': 17, 'after': 18, 'again': 19, 'agree': 20, 'ahead': 21, 'aid': 22, 'aim': 23, 'air': 24, 'alcohol': 25, 'aliens': 26, 'alive': 27, 'all': 28, 'allergies': 29, 'allow': 30, 'alls': 31, 'almost': 32, 'alone': 33, 'along': 34, 'already': 35, 'also': 36, 'aluminum': 37, 'always': 38, 'am': 39, 'america': 40, 'an': 41, 'and': 42, 'angry': 43, 'angryCOMMA': 44, 'animal': 45, 'animals': 46, 'annoying': 47, 'another': 48, 'answer': 49, 'any': 50, 'anybody': 51, 'anymore': 52, 'anyone': 53, 'anything': 54, 'anythings': 55, 'anywayCOMMA': 56, 'apartment': 57, 'appearances': 58, 'appeared': 59, 'apples': 60, 'approached': 61, 'approve': 62, 'arabic': 63, 'are': 64, 'arent': 65, 'around': 66, 'arrest': 67, 'arrive': 68, 'arrived': 6

In [125]:
# creating arrays of input and output data
encoder_input_data = np.zeros((len(lines.english), max_input_length), dtype='float32')
decoder_input_data = np.zeros((len(lines.deu), max_output_length), dtype='float32')

#one hot encoding the target data as Dense layer only gives one output through softmax layer
decoder_target_data = np.zeros((len(lines.deu), max_output_length, num_decoder_tokens))

In [126]:
print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)

(2931, 8)
(2931, 13)
(2931, 13, 2126)


In [127]:
# putting all the integer values in input, output data and target data
for i,(input_text, output_text) in enumerate(zip(lines.english, lines.deu)):
  for t, word in enumerate(input_text.split()):
    encoder_input_data[i,t] = input_token_index[word]
  for t,word in enumerate(output_text.split()):
    decoder_input_data[i,t] = output_token_index[word]
    # as decoder target data is ahead of decoder input data, it will not include start_ character(which will be given to decoder model at prediction)
    if t > 0:
      decoder_target_data[i,t-1,output_token_index[word]] = 1

In [128]:
print("encoder input data: ", encoder_input_data[1])
print('decoder input data: ', decoder_input_data[1])
print('decoder target data: ',decoder_target_data[1])
print('shape of sample decoder target data: ', decoder_target_data[1].shape)

encoder input data:  [599.   0.   0.   0.   0.   0.   0.   0.]
decoder input data:  [  0. 811.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
decoder target data:  [[0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
shape of sample decoder target data:  (13, 2126)


create and training the model

In [129]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model

In [130]:
# setting hyperparameters
embedding_size = 120
lstm_dim = 324

In [131]:
# building model for training stage
#encoder model

encoder_inputs = Input(shape=(None,))
en_x = Embedding(num_encoder_tokens, embedding_size)(encoder_inputs)
encoder = LSTM(lstm_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(en_x)
encoder_states = [state_h, state_c]

In [132]:
# decoder model

decoder_inputs = Input(shape=(None,))
final_dex = Embedding(num_decoder_tokens, embedding_size)(decoder_inputs)

decoder_lstm = LSTM(lstm_dim, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(final_dex, initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax') 

decoder_outputs = decoder_dense(decoder_outputs)

In [133]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [140]:
model.compile(optimizer='rmsprop',
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])

In [141]:
model.summary()
plot_model(model, to_file='Model_plot.png', show_shapes=True, show_layer_names=True)

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 input_12 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 embedding_7 (Embedding)        (None, None, 120)    167280      ['input_11[0][0]']               
                                                                                                  
 embedding_8 (Embedding)        (None, None, 120)    255120      ['input_12[0][0]']               
                                                                                            

In [142]:
model.summary()

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 input_12 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 embedding_7 (Embedding)        (None, None, 120)    167280      ['input_11[0][0]']               
                                                                                                  
 embedding_8 (Embedding)        (None, None, 120)    255120      ['input_12[0][0]']               
                                                                                            

In [143]:
r = model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=16, epochs=10, validation_split=0.10)

Epoch 1/10
165/165 [==============================] - 115s 544ms/step - loss: 2.0107 - accuracy: 0.1087 - val_loss: 2.9506 - val_accuracy: 0.0984
Epoch 2/10
165/165 [==============================] - 81s 488ms/step - loss: 1.8787 - accuracy: 0.1214 - val_loss: 2.9947 - val_accuracy: 0.0939
Epoch 3/10
165/165 [==============================] - 87s 528ms/step - loss: 1.7910 - accuracy: 0.1310 - val_loss: 2.9328 - val_accuracy: 0.1002
Epoch 4/10
165/165 [==============================] - 87s 526ms/step - loss: 1.7204 - accuracy: 0.1382 - val_loss: 2.9082 - val_accuracy: 0.0921
Epoch 5/10
165/165 [==============================] - 82s 499ms/step - loss: 1.6586 - accuracy: 0.1442 - val_loss: 3.0139 - val_accuracy: 0.0968
Epoch 6/10
165/165 [==============================] - 84s 507ms/step - loss: 1.5972 - accuracy: 0.1491 - val_loss: 2.9135 - val_accuracy: 0.0958
Epoch 7/10
165/165 [==============================] - 87s 530ms/step - loss: 1.5273 - accuracy: 0.1554 - val_loss: 2.9119 - val_a

In [144]:
#Inference Stage

#encoder model
encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.summary()

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, None)]            0         
                                                                 
 embedding_7 (Embedding)     (None, None, 120)         167280    
                                                                 
 lstm_4 (LSTM)               [(None, 324),             576720    
                              (None, 324),                       
                              (None, 324)]                       
                                                                 
Total params: 744,000
Trainable params: 744,000
Non-trainable params: 0
_________________________________________________________________


In [145]:
#decoder model
decoder_state_input_h = Input(shape=(lstm_dim,))
decoder_state_input_c = Input(shape=(lstm_dim,))
decoder_state_inputs = [decoder_state_input_h, decoder_state_input_c]

final_dex2 = Embedding(num_decoder_tokens, embedding_size)(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(final_dex2, initial_state=decoder_state_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model([decoder_inputs] + decoder_state_inputs, [decoder_outputs2] + decoder_states2)

In [146]:
# reversing the word index dictionary to get words from index values
reverse_input_char_index = dict((i,char) for char, i in input_token_index.items())
reverse_output_char_index = dict((i,char) for char, i in output_token_index.items())
print(reverse_input_char_index)
print(reverse_output_char_index)

{0: 'a', 1: 'about', 2: 'absolute', 3: 'absorbs', 4: 'accident', 5: 'ache', 6: 'across', 7: 'actress', 8: 'actually', 9: 'actuallyCOMMA', 10: 'add', 11: 'address', 12: 'admitted', 13: 'advance', 14: 'advantage', 15: 'advice', 16: 'afford', 17: 'afraid', 18: 'after', 19: 'again', 20: 'agree', 21: 'ahead', 22: 'aid', 23: 'aim', 24: 'air', 25: 'alcohol', 26: 'aliens', 27: 'alive', 28: 'all', 29: 'allergies', 30: 'allow', 31: 'alls', 32: 'almost', 33: 'alone', 34: 'along', 35: 'already', 36: 'also', 37: 'aluminum', 38: 'always', 39: 'am', 40: 'america', 41: 'an', 42: 'and', 43: 'angry', 44: 'angryCOMMA', 45: 'animal', 46: 'animals', 47: 'annoying', 48: 'another', 49: 'answer', 50: 'any', 51: 'anybody', 52: 'anymore', 53: 'anyone', 54: 'anything', 55: 'anythings', 56: 'anywayCOMMA', 57: 'apartment', 58: 'appearances', 59: 'appeared', 60: 'apples', 61: 'approached', 62: 'approve', 63: 'arabic', 64: 'are', 65: 'arent', 66: 'around', 67: 'arrest', 68: 'arrive', 69: 'arrived', 70: 'art', 71: 'a

In [147]:
# function to predict translation
def decode_seq(input_seq):
  state_values = encoder_model.predict(input_seq)

  target_seq = np.zeros((1,2))

  target_seq[0,0] = output_token_index['START_']

  stop_condition = False
  decoded_sentence = ''

  while not stop_condition:
    output_tokens, h, c = decoder_model.predict([target_seq] + state_values)

    sampled_token_index = np.argmax(output_tokens[0,0,:])
    sampled_char = reverse_output_char_index[sampled_token_index]

    decoded_sentence += ' ' + sampled_char

    if(sampled_char == '_END' or len(decoded_sentence) >100 ):
      stop_condition = True

    target_seq = np.zeros((1,2))
    target_seq[0,0] = sampled_token_index

    state_values = [h,c] 

  return decoded_sentence

In [151]:
#used for Compiles a function into a callable TensorFlow graph. (deprecated arguments) 
#(deprecated arguments)retracing the agrument correctly
tf.function(
    func=None,
    input_signature=None,
    autograph=True,
    jit_compile=None,
    reduce_retracing=False,
    experimental_implements=None,
    experimental_autograph_options=None,
    experimental_relax_shapes=None,
    experimental_compile=None,
    experimental_follow_type_hints=None
) 
tf.types.experimental.GenericFunction


tensorflow.python.types.core.GenericFunction

In [152]:
# testing the model for a sample from existing data
for seq_index in [23,4,67,1234,345]:
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_seq(input_seq)
    print('-')
    print('Input sentence:', lines.english[seq_index])
    print('Decoded sentence:', decoded_sentence)

1/1 [==============================] - 0s 172ms/step
-
Input sentence: hurry
Decoded sentence:  warte _END
1/1 [==============================] - 0s 141ms/step
-
Input sentence: run
Decoded sentence:  warte _END
1/1 [==============================] - 0s 141ms/step
-
Input sentence: i care
Decoded sentence:  habe _END
1/1 [==============================] - 0s 141ms/step
-
Input sentence: arent you canadian
Decoded sentence:  bist nicht _END
1/1 [==============================] - 0s 141ms/step
-
Input sentence: catch me
Decoded sentence:  warte _END
